In [235]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from bs4 import BeautifulSoup
import lxml
import requests
import unicodedata

In [236]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=(FutureWarning))

In [237]:
fm_df = pd.read_csv("players_db/fm23/uluta_fm23data.csv")

In [238]:
fm_df.columns = [col.strip() for col in fm_df.columns]

for col in fm_df.select_dtypes("object").columns:
   fm_df[col] = fm_df[col].apply(lambda x: x.strip())

In [239]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best Role', 'Best Duty',
       'Best Pos', 'Acc', 'Aer', 'Agg', 'Agi', 'Ant', 'Bal', 'Bra', 'Cmd',
       'Com', 'Cmp', 'Cnt', 'Cor', 'Cro', 'Dec', 'Det', 'Dri', 'Ecc', 'Fin',
       'Fir', 'Fla', 'Fre', 'Han', 'Hea', 'Jum', 'Kic', 'Ldr', 'Lon', 'L Th',
       'Mar', 'Nat .1', 'OtB', '1v1', 'Pac', 'Pas', 'Pen', 'Pos', 'Pun', 'Ref',
       'TRO', 'Sta', 'Str', 'Tck', 'Tea', 'Tec', 'Thr', 'Vis', 'Wor'],
      dtype='object')

In [240]:
# Assume df is your DataFrame
new_column_names = {
    'Acc': 'Acceleration',
    'Aer': 'Aerial_Reach',
    'Agg': 'Aggression',
    'Agi': 'Agility',
    'Ant': 'Anticipation',
    'Bal': 'Balance',
    'Bra': 'Bravery',
    'Cmd': 'Command_of_Area',
    'Com': 'Communication',
    'Cmp': 'Composure',
    'Cnt': 'Concentration',
    'Cor': 'Corners',
    'Cro': 'Crossing',
    'Dec': 'Decisions',
    'Det': 'Determination',
    'Dri': 'Dribbling',
    'Ecc': 'Eccentricity',
    'Fin': 'Finishing',
    'Fir': 'First_Touch',
    'Fla': 'Flair',
    'Fre': 'Free_Kick_Taking',
    'Han': 'Handling',
    'Hea': 'Heading',
    'Jum': 'Jumping_Reach',
    'Kic': 'Kicking',
    'Ldr': 'Leadership',
    'Lon': 'Long_Shots',
    'L Th': 'Long_Throws',
    'Mar': 'Marking',
    'Nat .1': 'Natural_Fitness',
    'OtB': 'Off_the_Ball',
    '1v1': 'One_on_Ones',
    'Pac': 'Pace',
    'Pas': 'Passing',
    'Pen': 'Penalty_Taking',
    'Pos': 'Positioning',
    'Pun': 'Punching',
    'Ref': 'Reflexes',
    'TRO': 'Rushing_Out',
    'Sta': 'Stamina',
    'Str': 'Strength',
    'Tck': 'Tackling',
    'Tea': 'Teamwork',
    'Tec': 'Technique',
    'Thr': 'Throwing',
    'Vis': 'Vision',
    'Wor': 'Work_Rate'
}

# new_column_names = {key: value.lower() for key, value in new_column_names.items()}
fm_df.rename(columns=new_column_names, inplace=True)

In [241]:
gk_attributes = [ "Aerial_Reach","Command_of_Area", "Communication","Eccentricity", "First_Touch", "Handling", "Kicking", "One_on_Ones", "Punching", "Reflexes", "Rushing_Out", "Throwing", "Passing" ]
fm_attributes = list(new_column_names.values())

fm_df[fm_attributes] = (fm_df[fm_attributes] * 4.95).astype(int)

In [242]:
fm_df['DoB'][0]

'14/8/1984 (37 years old)'

In [243]:
convert_date_format = lambda x: pd.to_datetime(x, format='%d/%m/%Y').strftime('%Y-%m-%d')
fm_df['DoB'] = fm_df['DoB'].apply(lambda x: x.split(" ")[0])
fm_df['DoB'] = fm_df['DoB'].apply(convert_date_format)

In [244]:
fm_df['DoB'][0]

'1984-08-14'

In [245]:
fm_df['Height'] = fm_df['Height'].apply(lambda x: x.split(" ")[0])

In [246]:
fm_df['Club_id'] = pd.Categorical(fm_df['Club']).codes
fm_df['League_id'] = pd.Categorical(fm_df['Division']).codes

In [247]:
fm_df = fm_df[fm_df['Inf'] != 'Yth']

In [248]:
fm_df = fm_df.rename(columns={
                        "Best Pos": "Best_Pos",
                        "Best Role": "Best_Role"
                    })

<br><br><br><br>
# Merged_DF

In [249]:
selected_leagues = ['English Premier Division',
 'Ligue 1 Uber Eats',
 'Spanish First Division',
 'Bundesliga',
 'Italian Serie A',
 'Eredivisie',
 'Portuguese Premier League',
 'Turkish Super League']

In [250]:
fm_df = fm_df[fm_df.Division.isin(selected_leagues)]

In [251]:
fm_df.columns

Index(['UID', 'Inf', 'Name', 'DoB', 'Nat', 'Division', 'Club', 'Based',
       'Preferred Foot', 'Right Foot', 'Left Foot', 'Position', 'Height',
       'Weight', 'Age', 'Transfer Value', 'Wage', 'AT Apps', 'AT Gls', 'Team',
       'Caps', 'Yth Apps', 'Style', 'Rc Injury', 'Best_Role', 'Best Duty',
       'Best_Pos', 'Acceleration', 'Aerial_Reach', 'Aggression', 'Agility',
       'Anticipation', 'Balance', 'Bravery', 'Command_of_Area',
       'Communication', 'Composure', 'Concentration', 'Corners', 'Crossing',
       'Decisions', 'Determination', 'Dribbling', 'Eccentricity', 'Finishing',
       'First_Touch', 'Flair', 'Free_Kick_Taking', 'Handling', 'Heading',
       'Jumping_Reach', 'Kicking', 'Leadership', 'Long_Shots', 'Long_Throws',
       'Marking', 'Natural_Fitness', 'Off_the_Ball', 'One_on_Ones', 'Pace',
       'Passing', 'Penalty_Taking', 'Positioning', 'Punching', 'Reflexes',
       'Rushing_Out', 'Stamina', 'Strength', 'Tackling', 'Teamwork',
       'Technique', 'Throwing', 

In [252]:
for leg in selected_leagues:
    q = fm_df.query(f"Division == '{leg}'")["Club"].value_counts()
    print(leg , "------", len(q.to_list()))

English Premier Division ------ 20
Ligue 1 Uber Eats ------ 20
Spanish First Division ------ 20
Bundesliga ------ 18
Italian Serie A ------ 20
Eredivisie ------ 18
Portuguese Premier League ------ 18
Turkish Super League ------ 19


In [253]:
fm_df.query(f"Division == 'Turkish Super League'")["Club"].value_counts()

Club
Kayserispor         41
Beşiktaş            41
Trabzonspor         40
Adana D.S.          39
Fenerbahçe          37
Ankaragücü          37
İstanbulspor        36
Galatasaray         35
Hatayspor           35
Kasımpaşa           35
Antalyaspor         34
Alanyaspor          34
Başakşehir FK       34
Sivasspor           33
Giresunspor         33
Gaziantep FK        32
Fatih Karagümrük    32
Ümraniyespor        32
Konyaspor           31
Name: count, dtype: int64

In [254]:
# fm_df.query(f"Club == 'Adana D.S.'").head()

<br><br><br><br>
# Team Names Converter from fminside.com

In [255]:
url_df = fm_df.drop_duplicates(subset=["Club"], keep="first")
url_df.loc[:, 'UID'] = url_df['UID'].apply(lambda text: str(text))
url_df = url_df[["Club", "UID"]]

In [256]:
def Player_to_URL(player_id):
    slug = player_id + "-x-x"
    return f"https://fminside.net/players/3-fm-23/{slug}"

In [257]:
def Convert_Team_Name_From_URL(player_id, club):
    url = Player_to_URL(player_id)
    req = requests.get(url)
    html = req.content
    soup = BeautifulSoup(html, "lxml")
    club_name_element = soup.find('div', class_='meta').find('ul').find_all('li')[0].text
    club_name = club_name_element.strip()
    return {"fm":club,"normal":club_name}

In [258]:
team_mapping_array = []
raw_names = url_df.to_dict()
uids = url_df["UID"]
clubs = url_df["Club"]
clubs_array = clubs.to_list()

In [259]:
# for uid,club in zip(uids,clubs):
#     new_entry = Convert_Team_Name_From_URL(uid, club)
#     print(new_entry)
#     team_mapping_array.append(new_entry)

In [260]:
fm_df.query("Name == 'Joshua Kimmich'")

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,Stamina,Strength,Tackling,Teamwork,Technique,Throwing,Vision,Work_Rate,Club_id,League_id
42332,92039023,,Joshua Kimmich,1995-02-08,GER,Bundesliga,FC Bayern,Germany (Bundesliga),Right,Very Strong,...,99,54,59,84,79,4,89,94,3704,185


In [261]:
fminside_normal_names = pd.read_csv("players_db/fm23/fminside_team_names.csv")
# merged_df = pd.merge(fm_df, fminside_normal_names, how='inner', left_on='Club', right_on='fm')
# merged_df['Club'] = merged_df['normal'].where(merged_df['Club'] == merged_df['fm'], merged_df['Club'])
# # merged_df.drop(['fm', 'normal'], axis=1, inplace=True)
# merged_df = merged_df[merged_df.fm.isin(clubs_array)]

In [262]:
fminside_club_names = []
for index, row in fminside_normal_names.iterrows():
  x_fm = row["fm"]
  x_normal = row["normal"]
  club_dict = {"fm": x_fm, "normal": x_normal}
  fminside_club_names.append(club_dict)

In [263]:
def Rename_Club(club_name):
    matched_club = ""
    for f in fminside_club_names:
        if f["fm"] == club_name:
            return f["normal"]

In [264]:
fm_df['Club'] = fm_df["Club"].apply(lambda x: Rename_Club(x))

In [265]:
fm_df.query("Name == 'Joshua Kimmich'")

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,Stamina,Strength,Tackling,Teamwork,Technique,Throwing,Vision,Work_Rate,Club_id,League_id
42332,92039023,,Joshua Kimmich,1995-02-08,GER,Bundesliga,FC Bayern,Germany (Bundesliga),Right,Very Strong,...,99,54,59,84,79,4,89,94,3704,185


<br><br><br><br>
# Position Styling

In [266]:
position_mapping = {
    'M (C)': 'MC',
    'D (C)': 'DC',
    'GK': 'GK',
    'ST (C)': 'ST',
    'AM (L)': 'AML',
    'AM (R)': 'AMR',
    'D (R)': 'DR',
    'D (L)': 'DL',
    'DM': 'DM',
    'AM (C)': 'AMC',
    'M (L)': 'ML',
    'M (R)': 'MR',
    'WB (L)': 'WBL',
    'WB (R)': 'WBR'
}

In [267]:
def remove_parentheses(text):
    return text.replace("(", "").replace(")", "")

In [268]:
def pos_map(position):
    return position_mapping[position]

In [269]:
fm_df['Best_Pos'].value_counts()

Best_Pos
M (C)     977
D (C)     938
AM (L)    738
GK        704
ST (C)    648
AM (R)    555
D (R)     521
D (L)     475
DM        388
AM (C)    180
M (L)     130
M (R)     116
WB (R)     29
WB (L)     23
Name: count, dtype: int64

In [270]:
fm_df['Position'].value_counts()

Position
D (C)                    749
GK                       704
DM, M (C)                600
ST (C)                   519
M/AM (C)                 279
                        ... 
D (R), WB/M (RL)           1
D (R), WB (RL), M (R)      1
D (RLC), WB/M (R)          1
WB/M (RL)                  1
D/AM (R)                   1
Name: count, Length: 224, dtype: int64

In [271]:
def Multiple_Position_Beautifier(position_value):
    x = ""
    positions = position_value.split(",")
    for p in positions:
        p = p.strip()
        if p in position_mapping.keys():
            x += f"{position_mapping[p]},"
        elif "/" in p:
            slashed = p.split(" ")
            locations = slashed[0].split("/")
            sides = list(remove_parentheses(slashed[1]))
            for loc in locations:
                for side in sides:
                    splitted_pos = loc+side
                    x += f"{splitted_pos},"
        else:
            slashed = p.split(" ")
            loc = slashed[0]
            sides = list(remove_parentheses(slashed[1]))
            for side in sides:
                splitted_pos = loc+side
                x += f"{splitted_pos},"
    return x[:-1]

In [272]:
fm_df['Position'] = fm_df['Position'].apply(lambda row: Multiple_Position_Beautifier(row))
fm_df['Best_Pos'] = fm_df['Best_Pos'].apply(lambda row: Multiple_Position_Beautifier(row))

# Player Role
fm_df["Player_Role"] = fm_df["Best_Pos"] + "_" + fm_df["Best_Role"]

In [273]:
position_order = ['GK', 'WBR', 'DR', 'DC', 'DL', 'WBL', 'DM',  
                  'MC', 'MR', 'AMC', 'ML', 'AMR', 'ST', 'AML']
def get_position_rank(position):
    return position_order.index(position) if position in position_order else len(position_order)
fm_df['Pos_Rank'] = fm_df['Best_Pos'].apply(get_position_rank)

In [274]:
fm_df.query("Club == 'Chaves'")[["Best_Pos"]]

,Best_Pos
168395,DC
168409,DC
168573,AML
168584,ST
168616,AML
168825,AML
169038,AMR
169074,GK
169161,MC
169202,ML


In [275]:
fm_df['Position'].value_counts()

Position
DC                  749
GK                  704
DM,MC               600
ST                  519
MC,AMC              279
                   ... 
DR,WBR,WBL,MR,ML      1
DR,WBR,WBL,MR         1
DR,DL,DC,WBR,MR       1
WBR,WBL,MR,ML         1
DR,AMR                1
Name: count, Length: 224, dtype: int64

In [276]:
fm_df['Best_Pos'].value_counts()

Best_Pos
MC     977
DC     938
AML    738
GK     704
ST     648
AMR    555
DR     521
DL     475
DM     388
AMC    180
ML     130
MR     116
WBR     29
WBL     23
Name: count, dtype: int64

<br><br><br><br>
# Players' Overall (GK Only)

In [277]:
def Dict_Generator(attrs):
    weights = (len(attrs)*"0.1,").split(",")[:-1]
    return {key: float(weights[i]) for i, key in enumerate(attrs)}

<h3 style="color:orange;">  Keeper</h3>

In [278]:
current_ability = "gk"
fm_df[current_ability] = fm_df[gk_attributes].mean(axis=1).astype(int)
fm_df[["Name", current_ability]].sort_values(current_ability, ascending=False).head()

,Name,gk
42335,Manuel Neuer,75
167966,Marc-André ter Stegen,74
151645,Ederson,72
151647,Alisson,72
167963,Jan Oblak,69


<br><br><br><br>
<h1 style="color:red;">  🥽TPR</h1>

<h3 style="color:aqua;">  Reputation</h3>

In [425]:
fm_df.shape

(6422, 97)

In [426]:
platinum_df = pd.read_csv('players_db/fm23/platinum22_fm23data.csv')[["Name","UID","Current reputation","Salary","Values"]]
inside_df = pd.read_csv('players_db/fm23/fm22_players_fminside.csv')[["Name","Unique_ID","Ability"]]

rep_df = pd.merge(fm_df, platinum_df, how='left', left_on='UID', right_on='UID', suffixes=('', '_PLAT'))
rep_df["Current reputation"].fillna(0, inplace=True)

In [427]:
rep_df.shape

(6422, 101)

In [420]:
rep_df = pd.merge(inside_df, platinum_df, how='left', left_on='Unique_ID', right_on='UID', suffixes=('', '_PLAT'))

In [421]:
filtered_df = rep_df[(rep_df['Current reputation'] == 6000) | (rep_df['Current reputation'] == 7000)]
filtered_df['Ability'].mean()

62.37965260545906

In [429]:
rep_df.query("Name == 'Mason Holgate'")

,UID,Inf,Name,DoB,Nat,Division,Club,Based,Preferred Foot,Right Foot,...,tpr_ML,tpr_MR,tpr_WBR,tpr_WBL,tpr_without_shifting,tpr_normalized,Name_PLAT,Current reputation,Salary,Values
3476,29128167,,Mason Holgate,1996-10-22,ENG,English Premier Division,Everton,England (Premier Division),Right,Very Strong,...,51,71,84,48,84,81,Mason Holgate,6497.0,84094.0,13261934.0


<br><br>
<h3 style="color:orange;">  Positional Ratings</h3>

In [287]:
positions = fm_df['Best_Pos'].unique().tolist()

In [288]:
Attribute_Position_Weights = pd.read_csv('players_db/fm23/Attribute_Position_Weights.csv',index_col=0)
Position_Position_Weights = pd.read_csv("players_db/fm23/Position_Position_Weights.csv", index_col=0)

In [490]:
def TPR_by_Position(row, position_weights_df=None, is_tpr_general=False, general_position=None):
    if is_tpr_general:
        position = general_position
    else:
        position = row["Best_Pos"] 
    rating_weights = position_weights_df[position].to_dict()
    rating_attrs = list(rating_weights.keys())
    tpr = (row[rating_attrs] * pd.Series(rating_weights)).sum()
    return int(tpr)

fm_df["tpr_general"] = fm_df.apply(lambda row: TPR_by_Position(row, Attribute_Position_Weights, \
                                                               is_tpr_general=True, general_position="MC"), axis=1)

In [314]:
fm_df["tpr"] = fm_df.apply(lambda row: TPR_by_Position(row, Attribute_Position_Weights), axis=1)

In [547]:
def Position_Regression(player_row, target_position):
    best_position = player_row.Best_Pos
    player_positions = player_row.Position
    tpr_value = player_row['tpr']
    
    coefficient = Position_Position_Weights.loc[best_position, target_position]
    base_value = int(coefficient / 20 * tpr_value)

    if target_position in player_positions.split(","):
        posdf_value = tpr_value - int((tpr_value - base_value) * 0.05)
        tpr_by_position_value = tpr_value - int((tpr_value - base_value) *  0.05)
        regression_value = max(posdf_value, tpr_by_position_value)
    else:
        regression_value = base_value

    return int(regression_value)

fm_df['tpr_Best'] = fm_df[tpr_cols].max(axis=1)
# fm_df['tpr_Star'] = "⭐⭐⭐⭐⭐"

In [548]:
for pos in positions:
    fm_df[f"tpr_{pos}"] = fm_df.apply(
        lambda row: Position_Regression(row, pos), axis=1
    )

In [549]:
tpr_cols = []
for col in positions:
    tpr_cols.append("tpr_" + col)

In [600]:
# Normalize tprs
def Normalize(X, min_val, max_val):
    min_col = X.min()
    max_col = X.max()
    standardized_value = (X - min_col) / (max_col - min_col) * (max_val - min_val) + min_val
    return standardized_value.astype(int)

for tpr_column in tpr_cols:
    fm_df[tpr_column] = Normalize(fm_df[tpr_column], 40, 99)
    
fm_df["tpr"] = Normalize(fm_df['tpr_Best'] , 40, 95)

# for pos in fm_df['Best_Pos'].unique():
#     group = fm_df#[fm_df['Best_Pos'] == pos]
#     min_tpr = group['tpr'].min()
#     max_tpr = group['tpr'].max()
#     fm_df.loc[fm_df['Best_Pos'] == pos, 'tpr_normalized'] = Normalize(group['tpr'], 40, 95)

fm_df["tpr_GK"] = fm_df.apply(lambda row: int(row["tpr_GK"]/2) if row.Best_Pos != "GK" else row["tpr_GK"], axis=1)

In [602]:
fm_df.sort_values("tpr_Best", ascending=False)[["Club","Name","Best_Pos","tpr_Best", \
                                                   "tpr"]].head(n=20)

,Club,Name,Best_Pos,tpr_Best,tpr
42332,FC Bayern,Joshua Kimmich,DM,99,95
151643,Manchester City,Bernardo Silva,AMR,99,95
167982,Real Madrid,David Alaba,DL,99,95
167965,Paris Saint-Germain,Sergio Ramos,DC,99,95
167958,FC Barcelona,Robert Lewandowski,ST,99,95
151638,Manchester City,Kevin De Bruyne,MC,99,95
151641,Tottenham Hotspur,Harry Kane,ST,99,95
151640,Liverpool,Mohamed Salah,AMR,99,95
151644,Liverpool,Virgil van Dijk,DC,99,95
151659,Manchester United,Bruno Fernandes,AMC,99,95


In [593]:
fm_df.sort_values(by="tpr_Best",ascending=False).head(500).Best_Pos.value_counts()

Best_Pos
GK     100
MC      84
DC      60
DL      56
DR      43
AML     42
DM      32
AMC     21
ST      19
AMR     17
ML      10
MR       7
WBL      6
WBR      3
Name: count, dtype: int64

In [580]:
fm_df.query("Club == 'Lazio'")[["Name","tpr","tpr_Best"]].head(n=5)

,Name,tpr,tpr_Best
73540,Sergej Milinković-Savić,4,97
73547,Ciro Immobile,4,94
73558,Luis Alberto,4,96
73562,Pedro,4,95
73571,Alessio Romagnoli,4,93


In [555]:
ex_row = fm_df.query("Name == 'Kylian Mbappé'")
ex_row.iloc[0][["tpr"] + tpr_cols].sort_values()

tpr_GK     23
tpr_DC     46
tpr_DM     48
tpr_DR     49
tpr_MC     56
tpr_WBR    58
tpr_DL     61
tpr_MR     62
tpr_AMC    71
tpr_WBL    72
tpr_ML     83
tpr        89
tpr_ST     94
tpr_AMR    96
tpr_AML    97
Name: 167957, dtype: object

In [556]:
ex_row = fm_df.query("Name == 'Lionel Messi'").iloc[0][["tpr"] + tpr_cols].sort_values()
ex_row

tpr_GK     23
tpr_DC     46
tpr_DL     48
tpr_DR     48
tpr_WBL    51
tpr_WBR    52
tpr_DM     56
tpr_MR     60
tpr_ML     72
tpr_MC     75
tpr_AML    79
tpr        90
tpr_ST     94
tpr_AMR    97
tpr_AMC    98
Name: 167956, dtype: object

# Matching fbref Player Names with FM Player Names

In [ ]:
from rapidfuzz import process
def find_best_match(name, choices):
    return process.extractOne(name, choices)

In [ ]:
top5_leagues = ['Ligue 1 Uber Eats', 'English Premier Division', 'Italian Serie A',
       'Spanish First Division', 'Bundesliga'] 

In [ ]:
url = "https://fbref.com/en/comps/Big5/2022-2023/passing/players/2022-2023-Big-5-European-Leagues-Stats"

In [ ]:
data = pd.read_html(url)

In [ ]:
data[0].columns = data[0].columns.get_level_values(1)

In [ ]:
data[0].head()

In [ ]:
fbref_players = data[0]["Player"].tolist()
fm_players = fm_df[fm_df.Division.isin(top5_leagues)]["Name"].tolist()

In [ ]:
for player_name in fm_players:
    answer, score, other = find_best_match(player_name, fbref_players)
    where_id = fm_df[fm_df.Name==player_name].iloc[0].name
    fm_df.at[where_id, 'fbref_player_name'] = answer

# CSV EXPORT

In [ ]:
fm_df.to_csv("players_db/fm23/fm23db_processed.csv", index=False)

In [ ]:
# fm_df.query("Name in 'Thiago Silva'").iloc[0].to_dict()